## Example use case of Octrac

In [1]:
import s3fs
import xarray as xr
import numpy as np
import pandas as pd
import dask.array as da
import ocetrac

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import warnings
warnings.filterwarnings('ignore')

### Import NOAA OISST v2.1 dataset and resample monthly means

In [2]:
endpoint_url = 'https://ncsa.osn.xsede.org'
fs_osn = s3fs.S3FileSystem(anon=True, client_kwargs={'endpoint_url': endpoint_url},)

path = "Pangeo/pangeo-forge/noaa_oisst/v2.1-avhrr.zarr"
ds = xr.open_zarr(fs_osn.get_mapper(path), consolidated=True, decode_timedelta=True).resample(time='MS').mean()
print(ds)

<xarray.Dataset>
Dimensions:  (time: 478, zlev: 1, lat: 720, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-10-01 ... 2021-06-01
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * zlev     (zlev) float32 0.0
Data variables:
    anom     (time, zlev, lat, lon) float32 dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
    err      (time, zlev, lat, lon) float32 dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
    ice      (time, zlev, lat, lon) float32 dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>
    sst      (time, zlev, lat, lon) float32 dask.array<chunksize=(1, 1, 720, 1440), meta=np.ndarray>


### Compute anomalies without the trend

In [14]:
dyr = ds.time.dt.year + ds.time.dt.month/12

In [21]:
sst = ds.sst.isel(zlev=0)
sst.shape

(478, 720, 1440)

In [22]:
# # 6 coefficient model is composed of the mean, trend, annual sine and cosine harmonics, & semi-annual sine and cosine harmonics
model = xr.DataArray([np.ones(len(dyr))] + [dyr-np.mean(dyr)] + [np.sin(2*np.pi*dyr)] + [np.cos(2*np.pi*dyr)] + [np.sin(4*np.pi*dyr)] + [np.cos(4*np.pi*dyr)],
                     dims=['coeff','time'], 
                     coords={'coeff':np.arange(1,7,1),'time':dyr}
                    ) 


# Take the pseudo-inverse of model to 'solve' least-squares problem
pmodel = xr.DataArray(np.linalg.pinv(model),
                      dims=['time', 'coeff'],
                      coords={'time':dyr, 'coeff':np.arange(1,7,1), }
                     ) 

# Transpose Arrays
model = model.transpose()
pmodel = pmodel.transpose() 

# resulting coefficients of the model
sst_mod = xr.DataArray(pmodel.dot(sst), dims=['coeff','lat','lon'], coords={'coeff':np.arange(1,7,1), 'lat':ds.lat.values, 'lon':ds.lon.values})  

# # Construction of the mean, trend, and seasonal cycle
# mean = xr.DataArray(model[:,0].dot(sst_mod[0,:,:]), dims=['time','lat','lon'], coords={'time':climatology.time, 'lat':climatology.lat, 'lon':climatology.lon})   
# trend = xr.DataArray(model[:,1].dot(sst_mod[1,:,:]), dims=['time','lat','lon'], coords={'time':climatology.time, 'lat':climatology.lat, 'lon':climatology.lon})    
# seas = xr.DataArray(model[:,2:].dot(sst_mod[2:,:,:]), dims=['time','lat','lon'], coords={'time':climatology.time, 'lat':climatology.lat, 'lon':climatology.lon})


In [ ]:
# compute anomalies (SSTa) by removing model coefficients without the trend
ssta_notrend = xr.DataArray((sst.values-model.dot(sst_mod)).values, dims=['time','lat','lon'], coords={'time':ds.time, 'lat':ds.lat.values, 'lon':ds.lon.values}) 



### Standardized SSTa by dividing by the monthly standard deviation
This step places equal variance on SSTa at all spatial points

In [ ]:
# Compute the monthly standard deviation
stdev = ssta_notrend.groupby(ssta_notrend.time.dt.month).std()

In [ ]:
# Divide by standard deviation
ssta_stn_notrend = ssta_notrend.groupby(ssta_notrend.time.dt.month) / stdev_notrend

### Use a threshold to find extreme anomalies

In [ ]:
# Use the 90th percentile as a threshold and find anomalies that exceed it. 

if ssta_stn_notrend.chunks:
    ssta_stn_notrend = ssta_stn_notrend.chunk({'time': -1})

threshold = ssta_stn_notrend.quantile(.9, dim=('time'))
features_notrend = ssta_stn_notrend.where(ssta_stn_notrend>=threshold, other=np.nan)


In [ ]:
%%time 
features_notrend.load()

In [ ]:
features_ssta.isel(time=-1).plot()

In [ ]:
# Binary conversion
binary_out = np.isfinite(features_ssta)
binary_out.isel(time=0).plot()

In [ ]:
mask = np.isfinite(ds.sst.isel(time=0, zlev=0))
mask.plot()

In [ ]:
%%time
Tracker = ocetrac.Tracker(binary_out, mask, radius=8, min_size_quartile=0.75, timedim='time', xdim='lon', ydim='lat', positive=True)
blobs = Tracker.track()

In [ ]:
blobs.attrs

In [ ]:
blobs

In [ ]:
blobs.sel(time=slice('01-01-2014', '04-01-2014')).plot(col='time', col_wrap=4)

In [ ]:
from matplotlib.colors import ListedColormap
maxl = int(np.nanmax(blobs.values))
cm = ListedColormap(np.random.random(size=(maxl, 3)).tolist())

plt.figure(figsize=(16,3))
ax1 = plt.subplot(121);blobs.isel(time=0).plot(cmap= cm)
mask.where(mask==0).plot.contourf(colors='k', add_colorbar=False); ax1.set_aspect('equal')
ax2 = plt.subplot(122); features_ssta.isel(time=0).plot(cmap='Reds', vmin=0, vmax=3);
mask.where(mask==0).plot.contourf(colors='k', add_colorbar=False); ax2.set_aspect('equal');

In [ ]:
id = 35
event = blobs.where(blobs==id, drop=True)
event.shape

In [ ]:
# Create intensity image for the Indian Ocean marine heatwave
event_intensity = ds.anom.isel(zlev=0).where((ds.time==event.time) & 
                                             (ds.lat==event.lat) & 
                                             (ds.lon==event.lon), 
                                             drop=True).load();
event_intensity = event_intensity.expand_dims(dim='intensity', axis=3)
events_contour = event.fillna(0)

plt.rc('font', size=12); plt.figure(figsize=(15,6));
for i in enumerate(range(1,len(event.time))):
    ax = plt.subplot(2,3,i[1],projection=ccrs.PlateCarree())
    event_intensity.isel(time=i[0], intensity=0).plot(transform=ccrs.PlateCarree(), vmin=-2, vmax=2,  
                                                      cmap='RdBu_r', extend='both', add_colorbar=True, add_labels=False)
    plt.title(event.isel(time=i[0]).time.values.astype('datetime64[D]'))
    ax.coastlines(resolution='110m', color='black', linewidth=1) 
    ax.add_feature(cfeature.LAND, facecolor='w');
    events_contour.isel(time=i[0]).plot.contour(levels=[34,35], transform=ccrs.PlateCarree(), colors='b', linewidths=4, add_colorbar=False, add_labels=False)

In [ ]:
mo = Tracker._morphological_operations()

In [ ]:
mo.isel(time=0).plot()